In [11]:
import os, time, re, random, glob, json, jieba
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
    TextGenerationPipeline
)

In [12]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [13]:
# root = "/Users/zeyesun/Documents"
# root = "/mnt/private-pa002-vol141056-prd"
root = "D://"

In [14]:
CLEAN_TEXT_PATTERN = re.compile(r"[\r\n]")

def clean_text(text):
    return CLEAN_TEXT_PATTERN.sub("", text)

# SFT

In [3]:
import sentencepiece
model_file = os.path.join(root,"Data", "models", "pangu-350M", "vocab.model")
sp = sentencepiece.SentencePieceProcessor()
sp.Load(model_file=model_file)

True

In [4]:
for i in range(10):
    print(sp.id_to_piece(i))

<unk>
<s>
</s>
▃
▂
<sep>
<pad>
<mask>
<eod>
<eot>


In [4]:
model_name_or_path = os.path.join(root, "Data", "models", "pangu-350M")
# model_name_or_path = os.path.join(root, "Data", "models", "pangu-2.6B")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer.add_special_tokens({
    'unk_token': "<unk>", 
    'eos_token': "<eot>", 
    'pad_token': "<pad>", 
    "sep_token": "<sep>"
})

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


0

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True, use_cache=False)
model.resize_token_embeddings(len(tokenizer.sp))
model.config.end_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# model.config.max_length_prompt = 200
model.to(device)
# print(model.device)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Some weights of GPTPanguForCausalLM were not initialized from the model checkpoint at D:\Data\models\pangu-350M and are newly initialized: ['transformer.h.23.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.15.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.bias', 'transformer.h.22.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.6.attn.bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.8.attn.bias', '

GPTPanguForCausalLM(
  (transformer): GPTPanguModel(
    (wte): Embedding(40000, 1024)
    (wpe): Embedding(1024, 1024)
    (wqe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPTPanguBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPTPanguAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTPanguMLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (c_proj): Linear(in_features=4096, out_features

### Data Processing

In [6]:
f = os.path.join(root, "Data", "raw", "baike_qa_train.json")
items = []
lens_prompt = []
lens_label = []
with open(f, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        item = json.loads(line.strip("\n"))
        prompt = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
        label = clean_text(item['answer'])
        items.append(item)
        lens_prompt.append(len(prompt))
        lens_label.append(len(label))
print(len(items))
print(np.percentile(lens_prompt, np.arange(90, 101)))
print(np.percentile(lens_label, np.arange(90, 101)))

1425170


In [43]:
i = 789
items[i]

{'qid': 'qid_7770627748113178417',
 'category': '生活-保健养生',
 'title': '减肥健身我要下载一套很好的有氧健身操，请问哪里有下载地址，谢谢！ ',
 'desc': '我要下载一套很好的有氧操，请问哪里有下载地址，谢谢！',
 'answer': '我看练太极拳就可以，该运动也是有氧健身的运动，长期锻炼还可以治疗很多慢性病。'}

In [44]:
items[i]['answer']

'我看练太极拳就可以，该运动也是有氧健身的运动，长期锻炼还可以治疗很多慢性病。'

### prediction

In [15]:
prefix = "模型回答："

In [6]:
checkpoint_files = os.path.join(root, "Data", "chatgpt", "output", "sft", "pangu-350M", "checkpoint-12000", "pytorch_model*.bin")
# checkpoint_files = os.path.join(root, "Data", "output", "sft", "pangu-2.6B", "checkpoint-9000", "pytorch_model*.bin")
checkpoints = glob.glob(checkpoint_files)
st = dict()
for checkpoint in checkpoints:
    st.update(torch.load(checkpoint, map_location="cpu"))
model.load_state_dict(st)

<All keys matched successfully>

In [7]:
text_generator = TextGenerationPipeline(model, tokenizer, device=device)

The model 'GPTPanguForCausalLM' is not supported for . Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBertForCausalLM', 'RoFormerForCausalLM', 'Speech2Text2ForCausalLM', 'TransfoXLLMHeadModel', 'TrOCRForCau

In [16]:
f = os.path.join(root, "Data", "raw", "baike_qa_train.json")
i = 0
prompts = []
prompts_processed = []
labels = []
with open(f, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        item = json.loads(line.strip("\n"))
        prompt = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
        label = clean_text(item['answer'])
        prompt_processed = prompt + tokenizer.sep_token + prefix
        prompts.append(prompt)
        prompts_processed.append(prompt_processed)
        labels.append(label)
        i += 1
        if i > 1000:
            break

In [17]:
num_return_sequences = 5
i = 200
j = 210
t1 = time.time()
results = text_generator(prompts_processed[i:j], max_length=200, num_return_sequences=num_return_sequences,
                         do_sample=True, top_k=50, temperature=10.0)
print(f"Finished prediction, time taken: {time.time()-t1}")

for prompt, res, label in zip(prompts[i:j], results[:(j-i)], labels[i:j]):
    print(f"prompt: {prompt}\nlabel: {label}")
    for k in range(num_return_sequences):
        model_answer = res[k]['generated_text'].split(prefix)[1].replace("<eot>", "").replace("<pad>", "")
        print(f"model answer-{k}: {model_answer}")
    print("\n\n")

Input length of input_ids is 456, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Finished prediction, time taken: 231.34307527542114
prompt: 三相四线直接接线式电表可否接互感器 
label: 可以的，但是必须要注意：应该提供三相电表内部电压线圈的供电，最简单的做法就是按互感器的“同名端”正确接入三相表后，再在三相线路每一相互感器的初极（即电路的一次侧）上，接一根线到互感器的次极（即接电表的二次侧）中接入电表的各个进线端的那根线上，这样提供了一个“电压源”给电表后，电度表就可以正常计量了。至于计量方法，则是电表的读数乘上互感器的“变流比”（即初极和次极电流比的值）就为实际用电的数值（KWh）。
model answer-0: 可接上导线(零相移一次半以上即中性两点接一线、)二次方都没有三相有一次三相三相零一组接线二回二(可以这样接接零二次)接RC4.因为三分明就是电流呀在接线盒把接地环接出来的二次半左右接一点线二回的二次可以带电阻,三相没一个中性,单相要用0,5电流绕三根粗线零线接接地,中性家用电器等电压一般就是3或者3.5。三连线法的变压器也不是中性地啊单相都中性吗那用铁氧性三相在小铜管前接三相有变压器啊接地地接中性三相?好专业、,求指导学习变压器和接网及配出接插片在此做一表理解我想说学习就做变压器
model answer-1: 电表安装完后的连接位置(线柱+相的距离应相等)如果安装后答案应选择A2那么线电流就该加至:LvV∴VF为5kb导体截量(电流I0100p的mA):pLw/mtxBVS电流率AsF+VT,(如lkwV+60mWLzwS5pH则电阻应该与50mA相应于90kWh/80nmolb相比有小量差别如电流是10的电阻才20u则要相差7(*2+/π*n)r是不可逆∴5A等于50+5w所以该种电流计算过程与A2相当选a应没超过5w电表只许输入6只线总也不够50
model answer-2: 这方面主要涉及仪表质量问题或者电流继电错误之说TT647-35F有几部分组成啊如果把电压接给5个的那就要三相交流电源串联在一电路中是无法保证PT的测量读数以及使用说明上面所述所提条件也包括实际意义所以只要知道所装测压部分电流信号的回路和电源也确定与电流相合便知会发生多大值多少分之间是否相称为可根据这些可以大概进行推算也没有明显问题在电流输入正常范围以下就是比较危险值和安全标准测量比较正

In [29]:
prompt = "北大和清华哪个更好？"
prompt_processed = prompt + tokenizer.sep_token + prefix
num_return_sequences=5
res = text_generator(prompt_processed, max_length=200, num_return_sequences=num_return_sequences,
                         do_sample=True, top_k=50, temperature=5.0)
print(f"prompt: {prompt}")
for i in range(num_return_sequences):
    model_answer = res[i]['generated_text'].split(prefix)[1].replace("<eot>", "").replace("<pad>", "")
    # print(res)
    print(f"model answer-{i}: {model_answer}")

prompt: 北大和清华哪个更好？
model answer-0: 现在学经济又觉得工科没法学可怕...我们清华北之大类院校每年招收大约5000～60学样生啊什么叫名牌呀他们会出奖学金就有什么名?就等于在这里给清华提供物质生活条件你再能挣一个天大学位证书那可谓如沐雨而知啊再说了就是读的清华还差多少学历才能升大学不进三本博,而是进二本还是专科要学那么厚基础啊能读二所?要看情况一般一本和上大学差几千块^-.这差个1线有几把沾?说要读四年?就可以哎还差几万字的毕业证了还要靠父母拿这些吃低保还是啥好啊要去好些个211学校你就要先多考好多分要你现在考不可能,你也没那功夫给人学个
model answer-1: 要读清华最好;北大比北大还要差一点但是清华又以教育和科学为优势因此如果只是学科学,而清华还需要高考考什么来改变就没有任何理由选这一题吧就这个问题就想选对方法最爽快些但高考不能就选择个方法就不能全掌握嘛希望您早日采纳...谢谢支持吧.您这是对北大文科生的不公正提问建议到这个网站首页,提问回答可以查看并与朋友提问相待了您若满意对其他还需要问题可以提出[非常抱歉需要时间发消息,或用百度搜号码搜到了给个提.的.或问QQ@1811447858请见本人提问]:(不含学院名-北京化大学新闻出版.学校资料可以参看北大新闻网上。网址在新闻类的页面即可)另外谢谢
model answer-2: 看学生本身素质好才高清华才真特好。看学习好了学生自然也多高考多厉害!要真说你这位学生的话学习是好学校还不该你在这里强争啊强压是多没有什么值得学习的好^自己在心里好好比评嘛学校里学得要真不是一个学生学习到这还是学到这样自己保重为重!!你大学生好好努力学习学习努力你想好就做真的吗要说大学好不好不是你能力有差问题吗就不要整天的挑你学没学学那不如趁早说那该不会北大都是说北大你要真心觉得有一点的那学校很差因为它所讲的话是你没弄懂该信别人的该真不要看大学排名嘛排名好不懂别乱说!但真的能考多不错你想想那个学校好嘛在
model answer-3: 从地理范围的划分不同看来讲就是现在学校最中心最核心最具有规模大学:清华其次则外企最核心最有价值专业所在地域则是僻洋之处或远的学校!(如西安地区也比大学比较少和`大=省少但教育好?北大的话:1和人大很熟2、复旦老二在三里,所以很受力`但清华不那么能混:但老一就那么可进是

# Reward Model

In [9]:
model_name_or_path = "D:\\Data\\models\\pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_cache=False, trust_remote_code=True)
tokenizer.add_special_tokens({'unk_token': "<unk>",
                                  'bos_token': "<s>",
                                  'eos_token': "<eot>",
                                  'pad_token': "<pad>",
                                  "sep_token": "<sep>"})

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


0

In [10]:
max_length = 1024
text = "你好，你是谁"
# text = "<|startoftext|>" + text + "<|endoftext|>"
res = tokenizer(text, max_length=max_length, truncation="longest_first", 
          return_tensors="pt", add_special_tokens=False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SUNZEY~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.501 seconds.
Prefix dict has been built successfully.


In [19]:
res.keys()
# torch.cat((res['input_ids'], res['input_ids']), axis=1)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])